In [23]:
import xarray as xr
import numpy as np
import netCDF4 as nc

In [13]:
import os
import sys
import xarray as xr
import netCDF4 as nc

print("=== ДИАГНОСТИКА ===")
print("1. Текущая папка:", os.getcwd())
print("2. Файл существует:", os.path.exists('US-Baltimore_clean_observations_v1.nc'))
print("3. Python exe:", sys.executable)

# Проверяем оба способа
print("\n4. Проверка netCDF4:")
try:
    ds_nc = nc.Dataset('US-Baltimore_clean_observations_v1.nc', 'r')
    print("   ✅ netCDF4 видит файл")
    ds_nc.close()
except Exception as e:
    print("   ❌ netCDF4:", e)

print("\n5. Проверка xarray:")
try:
    ds_xr = xr.open_dataset(r"C:\US-Baltimore_clean_observations_v1.nc", engine='netcdf4')
    print("   ✅ xarray видит файл")
except Exception as e:
    print("   ❌ xarray:", e)

=== ДИАГНОСТИКА ===
1. Текущая папка: c:\Users\User\OneDrive\Документы\GitHub\turbulent-fluxes-urban-ml
2. Файл существует: True
3. Python exe: c:\Users\User\anaconda3\envs\python312\python.exe

4. Проверка netCDF4:
   ✅ netCDF4 видит файл

5. Проверка xarray:
   ✅ xarray видит файл


In [18]:
dataset = nc.Dataset('US-Baltimore_clean_observations_v1.nc', 'r')
print (dataset.dimensions)
print (list(dataset.variables.keys()))

{'time': "<class 'netCDF4.Dimension'>": name = 'time', size = 43824}
['time', 'SWdown', 'LWdown', 'Tair', 'Qair', 'PSurf', 'Rainf', 'Snowf', 'Wind_N', 'Wind_E', 'SWup', 'LWup', 'Qle', 'Qh', 'SoilTemp', 'Qtau', 'SWdown_qc', 'LWdown_qc', 'Tair_qc', 'Qair_qc', 'PSurf_qc', 'Rainf_qc', 'Snowf_qc', 'Wind_N_qc', 'Wind_E_qc', 'SWup_qc', 'LWup_qc', 'Qle_qc', 'Qh_qc', 'SoilTemp_qc', 'Qtau_qc']


In [24]:
dataset = xr.open_dataset(r"C:\US-Baltimore_clean_observations_v1.nc", engine='netcdf4') #только для чтения
print(dataset)

<xarray.Dataset> Size: 4MB
Dimensions:      (time: 43824)
Coordinates:
  * time         (time) datetime64[ns] 351kB 2002-01-01T05:00:00 ... 2007-01-...
Data variables: (12/30)
    SWdown       (time) float32 175kB ...
    LWdown       (time) float32 175kB ...
    Tair         (time) float32 175kB ...
    Qair         (time) float32 175kB ...
    PSurf        (time) float32 175kB ...
    Rainf        (time) float32 175kB ...
    ...           ...
    SWup_qc      (time) int8 44kB ...
    LWup_qc      (time) int8 44kB ...
    Qle_qc       (time) int8 44kB ...
    Qh_qc        (time) int8 44kB ...
    SoilTemp_qc  (time) int8 44kB ...
    Qtau_qc      (time) int8 44kB ...
Attributes: (12/24)
    title:                      Flux tower observations from US-Baltimore (af...
    summary:                    Quality controlled flux tower observations fo...
    sitename:                   US-Baltimore
    long_sitename:              Cub Hill, Baltimore, United States
    version:                

In [25]:
for var_name in ['Qh', 'Qle', 'Qtau', 'Tair', 'Wind_N', 'Wind_E']: #используемые предикторы
    if var_name in dataset.data_vars:
        data = dataset[var_name]
        missing_count = np.isnan(data).sum().item()
        missing_pct = (missing_count / data.size)*100
        print (f"{var_name}: {missing_count}/{data.size} пропусков({missing_pct:.2f}%)")

Qh: 14336/43824 пропусков(32.71%)
Qle: 24166/43824 пропусков(55.14%)
Qtau: 7615/43824 пропусков(17.38%)
Tair: 248/43824 пропусков(0.57%)
Wind_N: 7472/43824 пропусков(17.05%)
Wind_E: 7472/43824 пропусков(17.05%)


In [27]:
for var_name in ['Qh', 'Qle', 'Qtau', 'Tair', 'Wind_N', 'Wind_E']:
    data = dataset[var_name]
    missing_count = np.isnan(data).sum().item()
    data_interpolated = data.interpolate_na(dim='time', method='linear') #линейная интерполяция
    missing_after_linear = np.isnan(data_interpolated).sum().item()
    filled_count = missing_count - missing_after_linear
    print(f"{var_name}:")
    print(f"  Линейная интерполяция: заполнено {filled_count}")
    print(f"  Осталось после линейной: {missing_after_linear}")

    missing_after_seasonal = missing_after_linear

    if missing_after_linear > 0:
        data_interpolated = data_interpolated.ffill(dim='time', limit=24) #интерполяция вперед на сутки
        data_interpolated = data_interpolated.bfill(dim='time', limit=24) #назад
        missing_after_seasonal = np.isnan(data_interpolated).sum().item()
        filled_seasonal = missing_after_linear - missing_after_seasonal
        print(f"  Сезонное заполнение: заполнено {filled_seasonal}")
        print(f"  Осталось после сезонного: {missing_after_seasonal}")

    if missing_after_seasonal > 0:
        data_interpolated = data_interpolated.ffill(dim='time') #агрессивная интерполяция
        data_interpolated = data_interpolated.bfill(dim='time')
        missing_after_aggressive = np.isnan(data_interpolated).sum().item()
        filled_aggressive = missing_after_seasonal - missing_after_aggressive
        print(f"  Агрессивное заполнение: заполнено {filled_aggressive}")
        print(f"  Осталось после агрессивного: {missing_after_aggressive}")
    
    dataset[var_name] = data_interpolated
    
    final_missing = np.isnan(dataset[var_name]).sum().item() #файнал результат
    total_filled = missing_count - final_missing
    print(f"итог: заполнено {total_filled} из {missing_count} пропусков")
    
    if final_missing == 0:
        print("пропуски устранены")
    else:
        print(f"осталось пропусков: {final_missing}")

Qh:
  Линейная интерполяция: заполнено 14331
  Осталось после линейной: 5
  Сезонное заполнение: заполнено 5
  Осталось после сезонного: 0
итог: заполнено 14336 из 14336 пропусков
пропуски устранены
Qle:
  Линейная интерполяция: заполнено 23915
  Осталось после линейной: 251
  Сезонное заполнение: заполнено 29
  Осталось после сезонного: 222
  Агрессивное заполнение: заполнено 222
  Осталось после агрессивного: 0
итог: заполнено 24166 из 24166 пропусков
пропуски устранены
Qtau:
  Линейная интерполяция: заполнено 7610
  Осталось после линейной: 5
  Сезонное заполнение: заполнено 5
  Осталось после сезонного: 0
итог: заполнено 7615 из 7615 пропусков
пропуски устранены
Tair:
  Линейная интерполяция: заполнено 248
  Осталось после линейной: 0
итог: заполнено 248 из 248 пропусков
пропуски устранены
Wind_N:
  Линейная интерполяция: заполнено 7467
  Осталось после линейной: 5
  Сезонное заполнение: заполнено 5
  Осталось после сезонного: 0
итог: заполнено 7472 из 7472 пропусков
пропуски устра

In [ ]:
def clean_physical_limits(data, var_name):
    limits = {
        'Qh': (-500, 1000),
        'Qle': (-100, 1500),
        'Qtau': (0, 5)
    }
    if var_name in limits:
        min_val, max_val = limits[var_name]

        cleaned = np.where ((data >= min_val) & (data <= max_val), data, np.nan)
        return cleaned
    return data